# 🔍 Azure Document Extraction Pipeline
## Using Responses API with Structured Outputs

### Architecture
```
📄 Document (PDF/Image)
       │
       ▼
┌──────────────────────────┐
│  Mistral Document AI     │  ← OCR + Text Extraction
│  (mistral-ocr-2503)      │
└──────────────────────────┘
       │
       ▼ OCR Text + Original Image
┌──────────────────────────┐
│  GPT-5.1                 │  ← Structured Extraction (Responses API)
│  (Responses API +        │     with strict JSON Schema
│   Structured Outputs)    │
└──────────────────────────┘
       │
       ▼ Extraction Results
┌──────────────────────────┐
│  Azure AI Evaluation SDK │  ← Quality Evaluation
└──────────────────────────┘
```

## 1️⃣ Install Dependencies

In [1]:
#!pip install -q --upgrade openai azure-ai-evaluation azure-identity python-dotenv pydantic requests

## 2️⃣ Configuration

Create a `.env` file in the same directory with the following variables:

```env
# Mistral Document AI
MISTRAL_ENDPOINT=https://<your-resource>.services.ai.azure.com
MISTRAL_API_KEY=<your-api-key>
MISTRAL_MODEL=mistral-ocr-2503

# Azure OpenAI (Responses API)
# Found in: Foundry > Models + endpoints > Your deployment
AZURE_OPENAI_ENDPOINT=https://<your-resource>.cognitiveservices.azure.com
AZURE_OPENAI_API_KEY=<your-api-key>
AZURE_OPENAI_DEPLOYMENT=gpt-5.1

# Evaluation Model
EVAL_MODEL_DEPLOYMENT=gpt-4o
```

In [2]:
import os
import json
import base64
import re
import requests
from pathlib import Path
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from openai import OpenAI

from azure.ai.evaluation import (
    GroundednessEvaluator,
    RelevanceEvaluator,
    CoherenceEvaluator,
    F1ScoreEvaluator,
    SimilarityEvaluator
)

load_dotenv()

print("✅ Imports successful")

✅ Imports successful


In [3]:
# =============================================================================
# Section 2: Configuration
# =============================================================================

import os
from dataclasses import dataclass
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

@dataclass
class Config:
    """Configuration loaded from environment variables."""
    
    # Mistral Document AI Configuration
    mistral_endpoint: str = os.getenv("MISTRAL_ENDPOINT", "")
    mistral_api_key: str = os.getenv("MISTRAL_API_KEY", "")
    mistral_model: str = os.getenv("MISTRAL_MODEL", "mistral-document-ai-2505-2")
    
    # Azure OpenAI Configuration
    aoai_endpoint: str = os.getenv("AZURE_OPENAI_ENDPOINT", "")
    aoai_api_key: str = os.getenv("AZURE_OPENAI_API_KEY", "")
    aoai_deployment: str = os.getenv("AZURE_OPENAI_DEPLOYMENT", "gpt-51")
    
    # Evaluation Configuration
    eval_deployment: str = os.getenv("EVAL_MODEL_DEPLOYMENT", "")
    
    @property
    def aoai_base_url(self) -> str:
        """Base URL for Responses API."""
        endpoint = self.aoai_endpoint.rstrip("/")
        # Add /openai/v1/ path for Responses API
        if not endpoint.endswith("/openai/v1"):
            endpoint = f"{endpoint}/openai/v1/"
        return endpoint
    
    def validate(self) -> dict:
        """Validate configuration and return status."""
        status = {
            "mistral_configured": bool(self.mistral_endpoint and self.mistral_api_key),
            "aoai_configured": bool(self.aoai_endpoint and self.aoai_api_key),
            "eval_configured": bool(self.eval_deployment)
        }
        return status

# Initialize configuration
config = Config()
status = config.validate()

print("=" * 50)
print("Configuration Status")
print("=" * 50)
print(f"Mistral Document AI: {'✅ Configured' if status['mistral_configured'] else '❌ Missing'}")
print(f"  - Endpoint: {config.mistral_endpoint[:50]}..." if config.mistral_endpoint else "  - Endpoint: Not set")
print(f"  - Model: {config.mistral_model}")
print(f"\nAzure OpenAI: {'✅ Configured' if status['aoai_configured'] else '❌ Missing'}")
print(f"  - Endpoint: {config.aoai_endpoint[:50]}..." if config.aoai_endpoint else "  - Endpoint: Not set")
print(f"  - Base URL: {config.aoai_base_url}")
print(f"  - Deployment: {config.aoai_deployment}")
print(f"\nEvaluation: {'✅ Configured' if status['eval_configured'] else '⚠️ Optional - not set'}")
print("=" * 50)

Configuration Status
Mistral Document AI: ✅ Configured
  - Endpoint: https://felat-mgtjiioy-francecentral.cognitiveserv...
  - Model: mistral-document-ai-2505-2

Azure OpenAI: ❌ Missing
  - Endpoint: https://doc-processing-mistral-gpt-eval.cognitives...
  - Base URL: https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com/openai/v1/
  - Deployment: gpt-5.1

Evaluation: ✅ Configured


## 3️⃣ Define Your Extraction Schema

Customize these Pydantic models based on your document type.

In [4]:
from typing import Optional, List, Literal

class PersonInfo(BaseModel):
    """Person information extracted from document."""
    full_name: str = Field(description="Full name of the person")
    date_of_birth: Optional[str] = Field(default=None, description="Date of birth (YYYY-MM-DD format)")
    address: Optional[str] = Field(default=None, description="Full address")
    email: Optional[str] = Field(default=None, description="Email address")
    phone: Optional[str] = Field(default=None, description="Phone number")


class MonetaryAmount(BaseModel):
    """Monetary amount with currency."""
    amount: float = Field(description="Numeric amount")
    currency: str = Field(default="USD", description="Currency code (e.g., USD, EUR)")


class LineItem(BaseModel):
    """A line item from a document."""
    description: str = Field(description="Item description")
    quantity: Optional[float] = Field(default=None, description="Quantity")
    unit_price: Optional[float] = Field(default=None, description="Price per unit")
    total: Optional[float] = Field(default=None, description="Total amount for this line")


class DocumentExtraction(BaseModel):
    """Generic document extraction schema. Customize for your use case."""
    
    document_type: str = Field(description="Type of document (e.g., invoice, contract, receipt)")
    document_number: Optional[str] = Field(default=None, description="Document reference number")
    document_date: Optional[str] = Field(default=None, description="Document date (YYYY-MM-DD)")
    
    sender: Optional[PersonInfo] = Field(default=None, description="Sender/issuer information")
    recipient: Optional[PersonInfo] = Field(default=None, description="Recipient information")
    
    total_amount: Optional[MonetaryAmount] = Field(default=None, description="Total amount")
    line_items: List[LineItem] = Field(default_factory=list, description="List of line items")
    
    due_date: Optional[str] = Field(default=None, description="Due date (YYYY-MM-DD)")
    effective_date: Optional[str] = Field(default=None, description="Effective date (YYYY-MM-DD)")
    expiration_date: Optional[str] = Field(default=None, description="Expiration date (YYYY-MM-DD)")
    
    notes: Optional[str] = Field(default=None, description="Additional notes")
    confidence_score: float = Field(description="Extraction confidence (0-1)")


def make_schema_strict(schema: dict) -> dict:
    """
    Transform a Pydantic JSON schema to be compatible with Azure OpenAI Structured Outputs.
    
    Azure OpenAI strict mode requires:
    1. additionalProperties: false on all objects
    2. All properties must be in the 'required' array
    """
    schema = schema.copy()
    
    def process_object(obj: dict) -> dict:
        """Recursively process an object schema."""
        if not isinstance(obj, dict):
            return obj
            
        obj = obj.copy()
        
        # If this is an object type, add additionalProperties: false and make all fields required
        if obj.get("type") == "object" and "properties" in obj:
            obj["additionalProperties"] = False
            # Make all properties required
            obj["required"] = list(obj["properties"].keys())
            # Process nested properties
            for prop_name, prop_value in obj["properties"].items():
                obj["properties"][prop_name] = process_object(prop_value)
        
        # Process $defs
        if "$defs" in obj:
            for def_name, def_value in obj["$defs"].items():
                obj["$defs"][def_name] = process_object(def_value)
        
        # Process items in arrays
        if "items" in obj:
            obj["items"] = process_object(obj["items"])
        
        # Process anyOf
        if "anyOf" in obj:
            obj["anyOf"] = [process_object(item) for item in obj["anyOf"]]
        
        # Process $ref (no change needed, but process siblings)
        if "$ref" in obj:
            pass  # $ref objects don't need processing
            
        return obj
    
    return process_object(schema)


# Generate schema and make it strict for Azure OpenAI
_raw_schema = DocumentExtraction.model_json_schema()
EXTRACTION_SCHEMA = make_schema_strict(_raw_schema)

print("📋 Extraction Schema (Strict Mode for Azure OpenAI):")
print(json.dumps(EXTRACTION_SCHEMA, indent=2))

📋 Extraction Schema (Strict Mode for Azure OpenAI):
{
  "$defs": {
    "LineItem": {
      "description": "A line item from a document.",
      "properties": {
        "description": {
          "description": "Item description",
          "title": "Description",
          "type": "string"
        },
        "quantity": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "description": "Quantity",
          "title": "Quantity"
        },
        "unit_price": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "description": "Price per unit",
          "title": "Unit Price"
        },
        "total": {
          "anyOf": [
            {
              "type": "number"
            },
            {
              "typ

## 4️⃣ Document Utilities

In [5]:
def encode_file_to_base64(file_path: str) -> Tuple[str, str]:
    """Encode a file to base64. Returns (base64_string, mime_type)."""
    path = Path(file_path)
    
    mime_types = {
        ".pdf": "application/pdf",
        ".png": "image/png",
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".webp": "image/webp",
        ".tiff": "image/tiff",
        ".tif": "image/tiff"
    }
    
    mime_type = mime_types.get(path.suffix.lower(), "application/octet-stream")
    
    with open(path, "rb") as f:
        base64_content = base64.b64encode(f.read()).decode("utf-8")
    
    return base64_content, mime_type


def get_file_info(file_path: str) -> Dict[str, Any]:
    """Get basic file information."""
    path = Path(file_path)
    return {
        "name": path.name,
        "extension": path.suffix.lower(),
        "size_bytes": path.stat().st_size,
        "size_mb": round(path.stat().st_size / (1024 * 1024), 2)
    }


print("✅ Document utilities ready")

✅ Document utilities ready


## 5️⃣ Step 1: OCR with Mistral Document AI

In [6]:
class MistralOCR:
    """Client for Mistral Document AI on Azure."""
    
    def __init__(self, endpoint: str, api_key: str, model: str = "mistral-ocr-2503"):
        self.endpoint = endpoint.rstrip("/")
        self.api_key = api_key
        self.model = model
        
    def extract_text(self, document_base64: str, mime_type: str, include_images: bool = False) -> Dict[str, Any]:
        """Extract text from a document using Mistral OCR."""
        url = f"{self.endpoint}/providers/mistral/azure/ocr" # farid changed
        
        if "pdf" in mime_type:
            document_url = f"data:{mime_type};base64,{document_base64}"
            payload = {
                "model": self.model,
                "document": {"type": "document_url", "document_url": document_url},
                "include_image_base64": include_images
            }
        else:
            image_url = f"data:{mime_type};base64,{document_base64}"
            payload = {
                "model": self.model,
                "document": {"type": "image_url", "image_url": image_url},
                "include_image_base64": include_images
            }
        
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()
        
        return response.json()
    
    def extract_from_file(self, file_path: str, include_images: bool = False) -> Dict[str, Any]:
        """Extract from a file path."""
        base64_content, mime_type = encode_file_to_base64(file_path)
        return self.extract_text(base64_content, mime_type, include_images)
    
    def get_markdown_text(self, ocr_result: Dict[str, Any]) -> str:
        """Extract markdown text from OCR result."""
        pages = ocr_result.get("pages", [])
        return "\n\n".join([page.get("markdown", "") for page in pages])


ocr_client = MistralOCR(
    endpoint=config.mistral_endpoint,
    api_key=config.mistral_api_key,
    model=config.mistral_model
)

print(f"✅ Mistral OCR client initialized (model: {config.mistral_model})")

✅ Mistral OCR client initialized (model: mistral-document-ai-2505-2)


## 6️⃣ Step 2: Structured Extraction with GPT-5.1 (Responses API)

In [7]:
# =============================================================================
# Section 6: Structured Extraction with Azure OpenAI Responses API (Entra ID)
# =============================================================================

import json
from openai import OpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

class StructuredExtractor:
    """Extract structured data using Azure OpenAI Responses API with Entra ID."""
    
    def __init__(self, endpoint: str, deployment: str):
        """
        Initialize the extractor with Entra ID authentication.
        
        Args:
            endpoint: Azure OpenAI endpoint (e.g., 'https://xxx.cognitiveservices.azure.com/')
            deployment: Model deployment name
        """
        # Get token provider for Entra ID auth
        token_provider = get_bearer_token_provider(
            DefaultAzureCredential(),
            "https://cognitiveservices.azure.com/.default"
        )
        
        # Build base URL for Responses API
        endpoint = endpoint.rstrip("/")
        base_url = f"{endpoint}/openai/v1/"
        
        self.client = OpenAI(
            base_url=base_url,
            api_key=token_provider()  # Get the token
        )
        self.deployment = deployment
        print(f"✅ StructuredExtractor initialized with Entra ID")
        print(f"   Base URL: {base_url}")
        print(f"   Deployment: {deployment}")
    
    def extract(
        self,
        text: str,
        schema: dict,
        schema_name: str = "extraction",
        instructions: str = None,
        image_base64: str = None,
        image_mime_type: str = "image/png"
    ) -> dict:
        """Extract structured data from text using the Responses API."""
        if instructions is None:
            instructions = """You are a document processing expert. Extract information 
from the provided document according to the specified schema. Be precise and extract 
only information that is explicitly present in the document. If a field cannot be 
found, use null or an appropriate default value."""
        
        input_content = []
        
        if image_base64:
            input_content.append({
                "type": "input_image",
                "image_url": f"data:{image_mime_type};base64,{image_base64}"
            })
        
        input_content.append({
            "type": "input_text",
            "text": f"Document content:\n\n---\n{text}\n---\n\nExtract the information according to the defined schema."
        })
        
        response = self.client.responses.create(
            model=self.deployment,
            instructions=instructions,
            input=[{"role": "user", "content": input_content}],
            text={
                "format": {
                    "type": "json_schema",
                    "name": schema_name,
                    "schema": schema,
                    "strict": True
                }
            },
            temperature=0
        )
        
        result = json.loads(response.output_text)
        result["_metadata"] = {
            "model": self.deployment,
            "response_id": response.id,
            "usage": {
                "input_tokens": response.usage.input_tokens if response.usage else None,
                "output_tokens": response.usage.output_tokens if response.usage else None
            }
        }
        
        return result

# Initialize - uses Entra ID, no API key needed
extractor = StructuredExtractor(
    endpoint="https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com",
    deployment="gpt-5.1"
)

✅ StructuredExtractor initialized with Entra ID
   Base URL: https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com/openai/v1/
   Deployment: gpt-5.1


## 7️⃣ Step 3: Quality Evaluation

In [8]:
class ExtractionValidator:
    """Custom validators for extraction quality."""
    
    @staticmethod
    def validate_date(date_str: Optional[str]) -> Tuple[bool, str]:
        if not date_str:
            return True, "No date provided (optional)"
        if re.match(r"^\d{4}-\d{2}-\d{2}$", date_str):
            return True, "Valid format"
        return False, f"Invalid format: {date_str}"
    
    @staticmethod
    def validate_amount(amount: Optional[float]) -> Tuple[bool, str]:
        if amount is None:
            return True, "No amount provided (optional)"
        if amount >= 0:
            return True, f"Valid amount: {amount}"
        return False, f"Negative amount: {amount}"
    
    @staticmethod
    def validate_confidence(score: Optional[float]) -> Tuple[bool, str]:
        if score is None:
            return False, "Missing confidence score"
        if 0 <= score <= 1:
            return True, f"Valid score: {score}"
        return False, f"Score out of range: {score}"
    
    def validate(self, extraction: Dict[str, Any]) -> Dict[str, Any]:
        """Run all validations."""
        results = {
            "validations": {},
            "total_checks": 0,
            "passed_checks": 0,
            "failed_checks": 0,
            "validation_score": 0.0
        }
        
        validations = [
            ("confidence_score", self.validate_confidence, extraction.get("confidence_score")),
            ("document_date", self.validate_date, extraction.get("document_date")),
            ("due_date", self.validate_date, extraction.get("due_date")),
            ("effective_date", self.validate_date, extraction.get("effective_date")),
            ("expiration_date", self.validate_date, extraction.get("expiration_date")),
        ]
        
        if extraction.get("total_amount"):
            validations.append(
                ("total_amount", self.validate_amount, extraction["total_amount"].get("amount"))
            )
        
        for name, validator, value in validations:
            is_valid, message = validator(value)
            results["validations"][name] = {"valid": is_valid, "message": message, "value": value}
            results["total_checks"] += 1
            if is_valid:
                results["passed_checks"] += 1
            else:
                results["failed_checks"] += 1
        
        if results["total_checks"] > 0:
            results["validation_score"] = results["passed_checks"] / results["total_checks"]
        
        return results


validator = ExtractionValidator()
print("✅ Extraction validator initialized")

✅ Extraction validator initialized


In [9]:
class QualityEvaluator:
    """Evaluate extraction quality using Azure AI Evaluation SDK."""
    
    def __init__(self, config: Config, tenant_id: str = None):
        """
        Initialize with Entra ID authentication.
        
        Args:
            config: Configuration object
            tenant_id: Optional Azure tenant ID. If your credential is from a different 
                      tenant than the Azure OpenAI resource, specify the resource's tenant ID.
        """
        from azure.identity import DefaultAzureCredential, AzureCliCredential
        
        # If tenant_id is specified, use it to get the correct credential
        if tenant_id:
            credential = DefaultAzureCredential(
                additionally_allowed_tenants=[tenant_id],
                exclude_shared_token_cache_credential=True
            )
            print(f"   Using tenant: {tenant_id}")
        else:
            # Try AzureCliCredential first as it's more predictable
            try:
                credential = AzureCliCredential()
                # Test the credential
                credential.get_token("https://cognitiveservices.azure.com/.default")
                print("   Using AzureCliCredential")
            except Exception:
                credential = DefaultAzureCredential()
                print("   Using DefaultAzureCredential")
        
        self.credential = credential
        
        # Azure AI Evaluation SDK only supports: gpt-35-turbo, gpt-4, gpt-4-turbo, gpt-4o, gpt-4o-mini
        # GPT-5.x is NOT supported. Use gpt-4o for evaluation.
        eval_deployment = "gpt-4o"  # Hardcoded - SDK doesn't support gpt-5.x
        
        # Use the same endpoint as StructuredExtractor
        eval_endpoint = "https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com"
        
        self.model_config = {
            "azure_endpoint": eval_endpoint,
            "azure_deployment": eval_deployment,
            "api_version": "2024-12-01-preview"
        }
        
        print(f"   Using endpoint: {eval_endpoint}")
        print(f"   Using deployment: {eval_deployment}")
        
        # Initialize evaluators with credential
        try:
            self.groundedness = GroundednessEvaluator(self.model_config, credential=credential)
            self.relevance = RelevanceEvaluator(self.model_config, credential=credential)
            self.coherence = CoherenceEvaluator(self.model_config, credential=credential)
            self.evaluators_initialized = True
        except Exception as e:
            print(f"   ⚠️ Could not initialize evaluators: {e}")
            self.evaluators_initialized = False
        
        self.validator = ExtractionValidator()
        
        print(f"✅ QualityEvaluator initialized with Entra ID + gpt-4o")
    
    def evaluate(self, source_text: str, extraction: Dict[str, Any]) -> Dict[str, Any]:
        """Run comprehensive evaluation."""
        results = {
            "ai_metrics": {},
            "validation_metrics": {},
            "overall_score": 0.0
        }
        
        if not self.evaluators_initialized:
            results["ai_metrics"]["error"] = "Evaluators not initialized - check authentication"
            results["validation_metrics"] = self.validator.validate(extraction)
            results["overall_score"] = results["validation_metrics"]["validation_score"]
            return results
        
        response_text = json.dumps(extraction, indent=2, ensure_ascii=False)
        query = "Extract structured data from the document."
        
        # AI metrics - using gpt-4o as judge
        # Groundedness is the most relevant for extraction: verifies data is in source
        try:
            results["ai_metrics"]["groundedness"] = self.groundedness(
                query=query, context=source_text, response=response_text
            )
        except Exception as e:
            error_msg = str(e)
            if "tenant" in error_msg.lower():
                results["ai_metrics"]["groundedness"] = {
                    "error": "Tenant mismatch - specify tenant_id in QualityEvaluator",
                    "details": error_msg[:200]
                }
            else:
                results["ai_metrics"]["groundedness"] = {"error": error_msg[:200]}
        
        try:
            results["ai_metrics"]["relevance"] = self.relevance(
                query=query, context=source_text, response=response_text
            )
        except Exception as e:
            results["ai_metrics"]["relevance"] = {"error": str(e)[:200]}
        
        try:
            results["ai_metrics"]["coherence"] = self.coherence(
                query=query, response=response_text
            )
        except Exception as e:
            results["ai_metrics"]["coherence"] = {"error": str(e)[:200]}
        
        # Custom validations
        results["validation_metrics"] = self.validator.validate(extraction)
        
        # Calculate overall score
        scores, weights = [], []
        
        for metric in ["groundedness", "relevance", "coherence"]:
            if metric in results["ai_metrics"] and "error" not in results["ai_metrics"][metric]:
                score = results["ai_metrics"][metric].get(metric, 0) / 5
                scores.append(score)
                weights.append(0.25)
        
        scores.append(results["validation_metrics"]["validation_score"])
        weights.append(0.15)
        
        if "confidence_score" in extraction:
            scores.append(extraction["confidence_score"])
            weights.append(0.1)
        
        if scores:
            results["overall_score"] = sum(s * w for s, w in zip(scores, weights)) / sum(weights)
        
        return results


# Initialize - if you get tenant mismatch errors, find your resource's tenant ID 
# in Azure Portal and pass it here:
# evaluator = QualityEvaluator(config, tenant_id="your-resource-tenant-id")
evaluator = QualityEvaluator(config)
print("✅ Quality evaluator initialized")

   Using AzureCliCredential
   Using endpoint: https://doc-processing-mistral-gpt-eval.cognitiveservices.azure.com
   Using deployment: gpt-4o
✅ QualityEvaluator initialized with Entra ID + gpt-4o
✅ Quality evaluator initialized


## 8️⃣ Complete Pipeline

In [10]:
def process_document(
    file_path: str,
    schema: Dict[str, Any] = EXTRACTION_SCHEMA,
    schema_name: str = "document_extraction",
    use_direct_pdf: bool = False,
    custom_instructions: Optional[str] = None,
    run_evaluation: bool = True,
    verbose: bool = True
) -> Dict[str, Any]:
    """Complete document processing pipeline."""
    
    results = {
        "file_info": get_file_info(file_path),
        "ocr_result": None,
        "extraction": None,
        "evaluation": None
    }
    
    if verbose:
        print(f"\n📄 Processing: {results['file_info']['name']}")
        print(f"   Size: {results['file_info']['size_mb']} MB")
    
    is_pdf = results["file_info"]["extension"] == ".pdf"
    
    if use_direct_pdf and is_pdf:
        if verbose:
            print("\n🔄 Mode: Direct PDF extraction via Responses API")
        
        pdf_base64, _ = encode_file_to_base64(file_path)
        extraction = extractor.extract_from_pdf(
            pdf_base64=pdf_base64,
            schema=schema,
            schema_name=schema_name,
            filename=results["file_info"]["name"],
            instructions=custom_instructions
        )
        results["extraction"] = extraction
        source_text = "[PDF processed directly by model]"
        
        if verbose:
            print(f"   ✅ Extraction complete")
        
    else:
        if verbose:
            print("\n🔄 Step 1: OCR with Mistral Document AI")
        
        ocr_result = ocr_client.extract_from_file(file_path)
        source_text = ocr_client.get_markdown_text(ocr_result)
        results["ocr_result"] = {
            "pages_processed": len(ocr_result.get("pages", [])),
            "text_length": len(source_text)
        }
        
        if verbose:
            print(f"   ✅ OCR complete ({results['ocr_result']['pages_processed']} pages)")
            print("\n🔄 Step 2: Structured extraction")
        
        extraction = extractor.extract(
            text=source_text,
            schema=schema,
            schema_name=schema_name,
            instructions=custom_instructions
        )
        results["extraction"] = extraction
        
        if verbose:
            print(f"   ✅ Extraction complete (confidence: {extraction.get('confidence_score', 'N/A')})")
    
    if run_evaluation:
        if verbose:
            print("\n🔄 Step 3: Quality evaluation")
        
        evaluation = evaluator.evaluate(source_text=source_text, extraction=extraction)
        results["evaluation"] = evaluation
        
        if verbose:
            print(f"   ✅ Evaluation complete (score: {evaluation['overall_score']:.1%})")
    
    return results


print("✅ Pipeline ready")
print("\n📝 Usage:")
print('   results = process_document("document.pdf")')
print('   results = process_document("document.pdf", use_direct_pdf=True)')

✅ Pipeline ready

📝 Usage:
   results = process_document("document.pdf")
   results = process_document("document.pdf", use_direct_pdf=True)


## 9️⃣ Example Usage

In [11]:
# Example: Process a document
# Uncomment and modify the path to your document

# results = process_document(
#     file_path="path/to/your/document.pdf",
#     use_direct_pdf=False,
#     verbose=True
# )

# print("\n📊 Extraction Results:")
# print(json.dumps(results["extraction"], indent=2, ensure_ascii=False))

In [12]:
def display_results(results: Dict[str, Any]):
    """Display results in a formatted way."""
    
    print("\n" + "="*60)
    print("📊 EXTRACTION RESULTS")
    print("="*60)
    
    extraction = results.get("extraction", {})
    
    print(f"\nDocument Type: {extraction.get('document_type', 'N/A')}")
    print(f"Document Number: {extraction.get('document_number', 'N/A')}")
    print(f"Document Date: {extraction.get('document_date', 'N/A')}")
    
    if extraction.get("total_amount"):
        amt = extraction["total_amount"]
        print(f"Total Amount: {amt.get('amount', 'N/A')} {amt.get('currency', '')}")
    
    print(f"Confidence Score: {extraction.get('confidence_score', 'N/A')}")
    
    if results.get("evaluation"):
        eval_data = results["evaluation"]
        
        print("\n" + "="*60)
        print("📈 EVALUATION RESULTS")
        print("="*60)
        
        for metric, data in eval_data.get("ai_metrics", {}).items():
            if "error" not in data:
                score = data.get(metric, "N/A")
                emoji = "✅" if isinstance(score, (int, float)) and score >= 4 else "⚠️"
                print(f"  {emoji} {metric.capitalize()}: {score}/5")
            else:
                print(f"  ❌ {metric.capitalize()}: Error")
        
        val_data = eval_data.get("validation_metrics", {})
        print(f"\nValidation Score: {val_data.get('validation_score', 0):.1%}")
        
        overall = eval_data.get("overall_score", 0)
        emoji = "🏆" if overall >= 0.9 else "✅" if overall >= 0.8 else "⚠️" if overall >= 0.7 else "❌"
        print(f"\n{emoji} OVERALL SCORE: {overall:.1%}")


def export_results(results: Dict[str, Any], output_path: str = "extraction_results.json"):
    """Export results to JSON file."""
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    print(f"✅ Results exported to: {output_path}")


print("✅ Display and export functions ready")

✅ Display and export functions ready


In [13]:
# Process the invoice
results = process_document("invoice.png", verbose=True)

# Display results
display_results(results)

# See raw extraction
print(json.dumps(results["extraction"], indent=2, ensure_ascii=False))


📄 Processing: invoice.png
   Size: 0.35 MB

🔄 Step 1: OCR with Mistral Document AI
   ✅ OCR complete (1 pages)

🔄 Step 2: Structured extraction
   ✅ Extraction complete (confidence: 0.96)

🔄 Step 3: Quality evaluation
   ✅ Evaluation complete (score: 84.6%)

📊 EXTRACTION RESULTS

Document Type: invoice
Document Number: 0324
Document Date: None
Total Amount: 500.0 USD
Confidence Score: 0.96

📈 EVALUATION RESULTS
  ✅ Groundedness: 4.0/5
  ✅ Relevance: 4.0/5
  ✅ Coherence: 4.0/5

Validation Score: 100.0%

✅ OVERALL SCORE: 84.6%
{
  "document_type": "invoice",
  "document_number": "0324",
  "document_date": null,
  "sender": {
    "full_name": "Paul J Gordon",
    "date_of_birth": null,
    "address": "2758 Paul Wayne Haggerty Road Kenner, LA 70062",
    "email": null,
    "phone": null
  },
  "recipient": null,
  "total_amount": {
    "amount": 500.0,
    "currency": "USD"
  },
  "line_items": [
    {
      "description": "Sed ut perspiciatis unde omnis iste lorem",
      "quantity": 5,


## 🎯 Summary

This notebook provides a document extraction pipeline using:

1. **Mistral Document AI** for OCR
2. **GPT-5.1 via Responses API** with strict JSON Schema validation
3. **Azure AI Evaluation SDK** for quality assessment

### To customize:
1. Modify the schema in Section 3 for your document type
2. Update validators in Section 7 for your rules
3. Use `custom_instructions` when calling `process_document()`